In [1]:
import matplotlib.pyplot as plt 
import tensorflow as tf
import numpy as np
import keras
from tensorflow.python.keras import regularizers
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Conv1D, Dropout, BatchNormalization, MaxPooling2D
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from numpy import asarray
from tensorflow.keras.applications.nasnet import preprocess_input
import random
from keras import layers
from tensorflow.keras.utils import plot_model
from sklearn import model_selection
from tensorflow.keras.initializers import glorot_uniform

In [2]:
tf.config.list_physical_devices(
    device_type=None
)
path = "machine-learning-in-science-2022/"

In [4]:
def read_dataset(img_path,label):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_png(img,channels=3)
    img = tf.image.resize(img,(224, 224))
    img = tf.cast(img, tf.float32)
    return img,label

In [5]:
def add_noise(img):
    noise = tf.random.normal(tf.shape(img),0,5)
    img += noise
    img = tf.clip_by_value(img, 0, 255)
    return img

In [6]:
df = pd.read_csv(path + 'training_norm.csv')
img_path = []
angle = []
for _,row in df.iterrows():
    img_path.append(path + 'training_data/training_data/' + str(int(row["image_id"])) + '.png')
    angle.append(row["angle"])
angle = np.array(angle)

In [7]:
img_path_train,img_path_val,angle_train,angle_val = model_selection.train_test_split(img_path,angle, test_size=0.2, train_size=0.8, random_state=None, shuffle=True, stratify=None)

In [8]:
ds_train = tf.data.Dataset.from_tensor_slices((img_path_train, angle_train))
ds_train = ds_train.map(read_dataset).map(lambda image, label: (tf.image.random_contrast(image, lower = 0.9, upper = 1), label)
                                                                ).map(lambda image, label: (tf.image.random_saturation(image, lower = 0.8, upper = 1.2),label)
                                                                     ).map(lambda image, label: (tf.image.random_brightness(image, 0.5 ,1.5),label)
                                                                          ).map(lambda image, label: (add_noise(image), label)
                                                                               ).shuffle(1000).batch(128).repeat(4)

ds_val = tf.data.Dataset.from_tensor_slices((img_path_val,angle_val))
ds_val = ds_val.map(read_dataset).batch(128)

Metal device set to: Apple M1 Max


2022-05-03 17:30:23.254381: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-03 17:30:23.254613: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [11]:
mobile = tf.keras.applications.MobileNetV3Large(input_shape=(224,224,3),weights = "imagenet")    
x = mobile.layers[-10].output
x = Dropout(0.2)(x)
x = Conv2D(2,(1,1),strides = (1, 1),kernel_initializer = glorot_uniform(seed=0))(x)
x = Flatten()(x)
angle1 = Dense(64, activation='relu', name='angle_hidden')(x)
angle_pred = Dense(1, activation = 'relu', name='angle')(angle1)
model = Model(inputs = mobile.input, outputs = [angle_pred]) #, speed_pred
for layer in model.layers[:-44]:
    layer.trainable = False 
model.compile(optimizer="adam", loss="mse", metrics = "mse")
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 224, 224, 3)  0           ['input_2[0][0]']                
                                                                                                  
 Conv (Conv2D)                  (None, 112, 112, 16  432         ['rescaling_1[0][0]']            
                                )                                                                 
                                                                                            

In [15]:
model.fit(ds_train,validation_data=ds_val,epochs = 5)

Epoch 1/5
348/348 [==============================] - 180s 507ms/step - loss: 7.7048e-04 - mse: 7.7048e-04 - val_loss: 0.0070 - val_mse: 0.0070
Epoch 2/5
348/348 [==============================] - 180s 510ms/step - loss: 7.4313e-04 - mse: 7.4313e-04 - val_loss: 0.0074 - val_mse: 0.0074
Epoch 3/5
348/348 [==============================] - 180s 508ms/step - loss: 6.4018e-04 - mse: 6.4018e-04 - val_loss: 0.0068 - val_mse: 0.0068
Epoch 4/5
348/348 [==============================] - 179s 505ms/step - loss: 6.3645e-04 - mse: 6.3645e-04 - val_loss: 0.0067 - val_mse: 0.0067
Epoch 5/5
348/348 [==============================] - 179s 505ms/step - loss: 5.8574e-04 - mse: 5.8574e-04 - val_loss: 0.0068 - val_mse: 0.0068


In [16]:
model.save("mobilenet_angle2")

INFO:tensorflow:Assets written to: mobilenet_angle2/assets


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)
